In [44]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

# pip install opencv-python
import cv2  

# pip install tensorflow
import tensorflow as tf
import random
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [36]:
# Set random seed for reproducibility
seed_value = 12345
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
random.seed(seed_value)

In [10]:
# Set the path to the dataset folders
train_data_path = "training_set"
test_data_path = "test_set"

In [11]:
# Set the input image dimensions
img_width, img_height = 150, 150

In [16]:
def load_dataset(data_path):
    images = []
    labels = []
    classes = ["cats", "dogs"]

    for class_name in classes:
        class_path = os.path.join(data_path, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            try:
                image = cv2.imread(image_path)
                image = cv2.resize(image, (img_width, img_height))
                images.append(image)
                labels.append(classes.index(class_name))
            except Exception as e:
                print(f"Error loading or resizing image: {image_path}")
                print(str(e))

    images = np.array(images)
    labels = to_categorical(labels)
    return images, labels

In [17]:
# Load and preprocess the training dataset
train_images, train_labels = load_dataset(train_data_path)
train_labels = train_labels[:, 1]  # Select only the second column (corresponding to "dogs")

Error loading or resizing image: training_set\cats\_DS_Store
OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Error loading or resizing image: training_set\dogs\_DS_Store
OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



In [18]:
train_images

array([[[[ 40,  44,  39],
         [ 43,  44,  40],
         [ 48,  45,  41],
         ...,
         [160, 178, 178],
         [176, 204, 208],
         [163, 197, 200]],

        [[ 40,  45,  40],
         [ 42,  43,  39],
         [ 46,  43,  39],
         ...,
         [158, 173, 174],
         [168, 194, 199],
         [164, 195, 200]],

        [[ 38,  42,  37],
         [ 37,  38,  34],
         [ 40,  37,  33],
         ...,
         [155, 167, 171],
         [163, 187, 193],
         [168, 197, 204]],

        ...,

        [[ 23,  22,  24],
         [ 20,  19,  21],
         [ 18,  17,  19],
         ...,
         [ 33,  38,  53],
         [ 43,  49,  62],
         [ 23,  29,  40]],

        [[ 29,  28,  30],
         [ 23,  22,  24],
         [ 20,  20,  22],
         ...,
         [ 25,  30,  45],
         [ 20,  26,  39],
         [ 30,  36,  47]],

        [[ 29,  29,  31],
         [ 22,  21,  23],
         [ 19,  18,  20],
         ...,
         [ 27,  34,  49],
        

In [19]:
train_labels

array([0., 0., 0., ..., 1., 1., 1.], dtype=float32)

In [37]:
# Split the training dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=seed_value)

In [51]:
# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Sigmoid activation for binary classification

In [52]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Binary cross-entropy loss

In [53]:
early_stopping_monitor = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(val_images, val_labels), callbacks=[early_stopping_monitor])

Epoch 1/10
129/129 [==============================] - 40s 308ms/step - loss: 6.3732 - accuracy: 0.5437 - val_loss: 0.6826 - val_accuracy: 0.5668
Epoch 2/10
129/129 [==============================] - 39s 306ms/step - loss: 0.6541 - accuracy: 0.6176 - val_loss: 0.6852 - val_accuracy: 0.5571
Epoch 3/10
129/129 [==============================] - 40s 307ms/step - loss: 0.5956 - accuracy: 0.6769 - val_loss: 0.7013 - val_accuracy: 0.6166
Epoch 4/10
129/129 [==============================] - 40s 306ms/step - loss: 0.5119 - accuracy: 0.7230 - val_loss: 0.7534 - val_accuracy: 0.6185
Epoch 5/10
129/129 [==============================] - 40s 308ms/step - loss: 0.4324 - accuracy: 0.7918 - val_loss: 0.8479 - val_accuracy: 0.6341
Epoch 6/10
129/129 [==============================] - 40s 309ms/step - loss: 0.3689 - accuracy: 0.8402 - val_loss: 1.1068 - val_accuracy: 0.6410
Epoch 7/10
129/129 [==============================] - 40s 309ms/step - loss: 0.2673 - accuracy: 0.8770 - val_loss: 1.1575 - val_ac

In [54]:
# Save model with best epoch/weights
model.save_weights("model.h5")

In [55]:
# Load and preprocess the test dataset
test_images, test_labels = load_dataset(test_data_path)
test_labels = test_labels[:, 1]  # Select only the second column (corresponding to "dog")

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

Error loading or resizing image: test_set\cats\_DS_Store
OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Error loading or resizing image: test_set\dogs\_DS_Store
OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

64/64 [==============================] - 4s 65ms/step - loss: 1.2876 - accuracy: 0.6352
Test Loss: 1.2875921726226807
Test Accuracy: 0.635195255279541
